In [1]:
## import Library ##
import os
import random
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from keras.models import Model
from keras import optimizers, applications
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input

In [2]:
## Seed ##
from tensorflow import set_random_seed
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    set_random_seed(0)
seed_everything()

In [3]:
## Load Data ##
train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [4]:
print('Number of train samples: ', train.shape[0])
print('Number of test samples: ', test.shape[0])
display(train.head())

In [5]:
## Model parameters ##
BATCH_SIZE = 4
EPOCHS = 10
WARMUP_EPOCHS = 2
LEARNING_RATE = 1e-4
WARMUP_LEARNING_RATE = 1e-3
HEIGHT = 512
WIDTH = 512
CANAL = 3
N_CLASSES = train['diagnosis'].nunique()
ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5

In [6]:
# Preprocecss data
train["id_code"] = train["id_code"].apply(lambda x: x + ".png")
test["id_code"] = test["id_code"].apply(lambda x: x + ".png")
train['diagnosis'] = train['diagnosis'].astype('str')
train.head()

In [7]:
## image augmentation ## 
train_datagen=ImageDataGenerator(rescale=1./255, 
                                 validation_split=0.2,
                                 horizontal_flip=True)

## Data generator ##
train_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/aptos2019-blindness-detection/train_images/",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    target_size=(HEIGHT, WIDTH),
    subset='training')

valid_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/aptos2019-blindness-detection/train_images/",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=BATCH_SIZE,
    class_mode="categorical",    
    target_size=(HEIGHT, WIDTH),
    subset='validation')

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size

In [8]:
## Model(InceptionResNetV2) ##
from keras.applications.inception_resnet_v2 import InceptionResNetV2
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionResNetV2(weights = None, include_top = False, input_tensor = input_tensor)
    base_model.load_weights('../input/tf-keras-pretrained-model-weights/No Top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

model = create_model(input_shape=(HEIGHT, WIDTH, CANAL), n_out=N_CLASSES)
# model.summary()

In [9]:
# ## Model(Xception) ##
# from keras.applications.xception import Xception
# def create_model(input_shape, n_out):
#     input_tensor = Input(shape=input_shape)
#     base_model = Xception(weights = None, include_top = False, input_tensor = input_tensor)
#     base_model.load_weights('../input/tf-keras-pretrained-model-weights/No Top/xception_weights_tf_dim_ordering_tf_kernels_notop.h5')
    
#     x = GlobalAveragePooling2D()(base_model.output)
#     x = Dropout(0.5)(x)
#     x = Dense(2048, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     final_output = Dense(n_out, activation='softmax', name='final_output')(x)
#     model = Model(input_tensor, final_output)
    
#     return model

# model = create_model(input_shape=(HEIGHT, WIDTH, CANAL), n_out=N_CLASSES)
# # model.summary()

In [10]:
## Training Top Layers(base model 제외) ## 
for layer in model.layers:
    layer.trainable = False

for i in range(-5, 0):
    model.layers[i].trainable = True

metric_list = ["accuracy"]
optimizer = optimizers.Adam(lr=WARMUP_LEARNING_RATE)
model.compile(optimizer=optimizer, loss="categorical_crossentropy",  metrics=metric_list)

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=WARMUP_EPOCHS,
                    verbose=1)


In [11]:
## Training Layers(complete model) ##
for layer in model.layers:
    layer.trainable = True

es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)
cp = ModelCheckpoint(filepath='checkpoint_xception.ckpt',save_best_only = True, save_weights_only=True,verbose=1)

callback_list = [es, rlrop, cp]
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
model.compile(optimizer=optimizer, loss="binary_crossentropy",  metrics=metric_list)

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=EPOCHS,
                    callbacks=callback_list,
                    verbose=1)